# Projet : Systeme de recommandation d'images

## Presentation

Dans ce projet, vous allez construire un **systeme de recommandation d'images** qui suggere des images aux utilisateurs en fonction de leurs preferences. Ce projet met en pratique toutes les competences acquises lors des seances pratiques : analyse de donnees, visualisation, regroupement, classification et apprentissage automatique.

**Duree** : 3 seances pratiques
**Taille de l'equipe** : 2-3 etudiants
**Livrables** :
1. Un notebook Jupyter (`Nom1_Nom2_[Nom3].ipynb`)
2. Un rapport de synthese de 4 pages (PDF)

---

## Objectifs d'apprentissage

En completant ce projet, vous serez capable de :
- Automatiser la collecte de donnees a partir de sources web
- Extraire et traiter les metadonnees d'images
- Appliquer des algorithmes de regroupement pour analyser les caracteristiques des images
- Construire des profils de preferences utilisateur
- Implementer un algorithme de recommandation
- Visualiser efficacement les donnees
- Ecrire des tests complets pour votre systeme

---

## Architecture du projet

Le systeme est compose de 7 taches interconnectees :

```
┌─────────────────────────────────────────────────────────────────┐
│               SYSTEME DE RECOMMANDATION D'IMAGES                 │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  ┌──────────────┐    ┌──────────────┐    ┌──────────────┐       │
│  │ 1. Collecte  │───▶│ 2. Etiquetage│───▶│ 3. Analyse   │       │
│  │ de donnees   │    │ & Annotation │    │ de donnees   │       │
│  └──────────────┘    └──────────────┘    └──────────────┘       │
│        │                    │                   │                │
│        ▼                    ▼                   ▼                │
│  ┌──────────────────────────────────────────────────────┐       │
│  │          Fichiers JSON (Stockage des metadonnees)     │       │
│  └──────────────────────────────────────────────────────┘       │
│        │                    │                   │                │
│        ▼                    ▼                   ▼                │
│  ┌──────────────┐    ┌──────────────┐    ┌──────────────┐       │
│  │ 4. Visuali-  │    │ 5. Systeme de│    │ 6. Tests     │       │
│  │ sation       │    │ recommandat. │    │              │       │
│  └──────────────┘    └──────────────┘    └──────────────┘       │
│                             │                                    │
│                             ▼                                    │
│                    ┌──────────────┐                              │
│                    │ 7. Rapport   │                              │
│                    │ de synthese  │                              │
│                    └──────────────┘                              │
└─────────────────────────────────────────────────────────────────┘
```

![Architecture](../../images/Project-Architecture.png "Architecture")

---

## Projet partie 1

---

### Tache 1 : Collecte de donnees

#### Objectif
Collecter au moins **100 images sous licence ouverte** avec leurs metadonnees.

#### Ce que vous devez faire

1. **Creer une structure de dossiers** :
   ```
   projet/
   ├── images/           # Images telechargees
   ├── data/             # Fichiers de metadonnees JSON
   └── projet.ipynb      # Votre notebook
   ```

2. **Trouver des sources d'images** (une ou plusieurs) :
   - [Wikimedia Commons](https://commons.wikimedia.org/) - Utilisez des requetes SPARQL (comme dans le TP 1)
   - [Unsplash API](https://unsplash.com/developers) - API gratuite pour des images de haute qualite
   - [Pexels API](https://www.pexels.com/api/) - Photos libres de droits
   - [Flickr API](https://www.flickr.com/services/api/) - Images Creative Commons

3. **Telecharger les images par programme** en utilisant les techniques du TP 1, Exercice 6

4. **Extraire et sauvegarder les metadonnees** de chaque image :
   - Nom du fichier image
   - Dimensions de l'image (largeur, hauteur)
   - Format du fichier (.jpg, .png, etc.)
   - Taille du fichier (en Ko)
   - URL source
   - Informations de licence
   - Donnees EXIF (si disponibles) : modele d'appareil photo, date de prise de vue, etc.

#### Resultat attendu
- Dossier `images/` contenant 100+ images
- `data/images_metadata.json` contenant les metadonnees de toutes les images

#### Conseils
- Utilisez `PIL` pour obtenir les dimensions de l'image
- Utilisez `os.path.getsize()` pour obtenir la taille du fichier
- Utilisez l'extraction EXIF (voir TP 2, Exercice 2)
- Stockez les metadonnees sous forme de liste de dictionnaires au format JSON

---

## Tâche 1 — Collecte de données : téléchargement des images
On récupère la liste des photos depuis l'API `jmail.world`, puis on télécharge les 100 premières dans `images/`.

In [2]:
import requests, time
import json
from pathlib import Path

# ── Config ─────────────────────────────────────────────────────
LIMIT = 100
API_URL = "https://jmail.world/api/photos?newOnly=false"
CDN_BASE = "https://assets.getkino.com/photos-deboned"
IMAGES_DIR = Path("images")
DATA_DIR = Path("data")
IMAGES_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR.mkdir(parents=True, exist_ok=True)

# ── 1. Récupérer la liste des photos depuis l'API ─────────────
session = requests.Session()
resp = session.get(API_URL, timeout=30)
resp.raise_for_status()
all_photos = resp.json()["photos"]

# Filtrer : garder seulement les images AVEC source_url
available_photos = [p for p in all_photos if p.get("source_url")]
print(f"API : {len(all_photos)} photos total, {len(available_photos)} avec source_url")

# ── 2. Télécharger jusqu'à avoir LIMIT images réussies ────────
downloaded = 0
successful_photos = []
attempted = 0

for photo in available_photos:
    if downloaded >= LIMIT:
        break
    
    attempted += 1
    filename = photo["original_filename"]
    dest = IMAGES_DIR / filename
    
    # Si l'image existe déjà, on la compte comme réussie
    if dest.exists():
        downloaded += 1
        successful_photos.append(photo)
        continue
    
    url = f"{CDN_BASE}/{filename}"
    try:
        r = session.get(url, stream=True, timeout=60)
        r.raise_for_status()
        with open(dest, "wb") as f:
            for chunk in r.iter_content(8192):
                f.write(chunk)
        downloaded += 1
        successful_photos.append(photo)
        if downloaded % 20 == 0:
            print(f"  [{downloaded}/{LIMIT} downloaded, {attempted} tried]…")
    except Exception as e:
        print(f"  ✗ {filename}: {e}")
    
    time.sleep(0.05)

# ── 3. Sauvegarder SEULEMENT les métadonnées des images téléchargées ──
metadata_file = DATA_DIR / "image_metadata.json"
with open(metadata_file, "w", encoding="utf-8") as f:
    json.dump(successful_photos, f, ensure_ascii=False, indent=2)

print(f"\n✓ {downloaded} images dans {IMAGES_DIR}/ (essayé {attempted} photos)")
print(f"✓ Métadonnées de {len(successful_photos)} images sauvegardées dans {metadata_file}")

API : 18316 photos total, 5527 avec source_url
  ✗ EFTA00004157-3.png: 404 Client Error: Not Found for url: https://assets.getkino.com/photos-deboned/EFTA00004157-3.png
  ✗ EFTA00004157-5.png: 404 Client Error: Not Found for url: https://assets.getkino.com/photos-deboned/EFTA00004157-5.png
  ✗ EFTA00004157-7.png: 404 Client Error: Not Found for url: https://assets.getkino.com/photos-deboned/EFTA00004157-7.png
  ✗ EFTA00005381-1.png: 404 Client Error: Not Found for url: https://assets.getkino.com/photos-deboned/EFTA00005381-1.png
  ✗ EFTA00005386-78.png: 404 Client Error: Not Found for url: https://assets.getkino.com/photos-deboned/EFTA00005386-78.png
  ✗ EFTA00005569-1.png: 404 Client Error: Not Found for url: https://assets.getkino.com/photos-deboned/EFTA00005569-1.png
  [20/100 downloaded, 26 tried]…
  [40/100 downloaded, 46 tried]…
  [60/100 downloaded, 66 tried]…
  [80/100 downloaded, 86 tried]…
  [100/100 downloaded, 106 tried]…

✓ 100 images dans images/ (essayé 106 photos)
✓ Mét

In [3]:
!pip install Pillow exifread

In [28]:
from PIL import Image
import json
import os
import pandas as pd

modeListe = []
tailleListe = []
sizeCategoryListe = []
orientationListe = []

# Load JSON data from a file
data = json.load(open("data/image_metadata.json"))

# Convert JSON data to a pandas DataFrame
dataframe = pd.json_normalize(data)

for index, row in dataframe.iterrows():
    
    image_path = "images/" + row["id"]
    
    if os.path.exists(image_path):
        with Image.open(image_path) as img:
            modeListe.append(img.mode)
            tailleListe.append(str(round(os.path.getsize(image_path) / 1024, 2)) + " ko")
            
            # Size classification based on maximum dimension
            width, height = img.size
            max_dimension = max(width, height)
            
            if max_dimension < 500:
                sizeCategoryListe.append("thumbnail")
            elif max_dimension <= 1500:
                sizeCategoryListe.append("medium")
            else:
                sizeCategoryListe.append("large")
            
            # Orientation detection
            if width > height:
                orientationListe.append("landscape")
            elif height > width:
                orientationListe.append("portrait")
            else:
                orientationListe.append("square")
    else:
        paletteListe.append("Image not found")
        modeListe.append("Image not found")
        tailleListe.append("Image not found")
        sizeCategoryListe.append("unknown")
        orientationListe.append("unknown")
        
dataframe["mode"] = modeListe
dataframe["size"] = tailleListe
dataframe["size_category"] = sizeCategoryListe
dataframe["orientation"] = orientationListe
dataframe.to_json(path_or_buf="data/metadata.json", orient="records", indent=1)


In [25]:
tag = ["album_photo", "extérieur", "illisible", "intérieur", "aJE", "jour", "nuit"]

photo_tags = [
    [tag[1], tag[4], tag[5]],
    [tag[1], tag[4], tag[5]],
    [tag[1], tag[4], tag[5]],
    [tag[1], tag[4], tag[5]],
    [tag[1], tag[4], tag[5]],
    [tag[1], tag[5]],
    [tag[3], tag[4]],
    [tag[3], tag[4]],
    [tag[3], tag[4]],
    [tag[3]],
    [tag[3]],
    [tag[3]],
    [tag[3], tag[4], tag[6]],
    [tag[3]],
    [tag[3]],
    [tag[3], tag[4]],
    [tag[3], tag[4]],
    [tag[3], tag[4]],
    [tag[3], tag[4], tag[6]],
    [tag[3], tag[4], tag[6]],
    [tag[1], tag[5]],
    [tag[3]],
    [tag[1], tag[5]],
    [tag[3]],
    [tag[3]],
    [tag[3]],
    [tag[3]],
    [tag[1], tag[5]],
    [tag[1], tag[4], tag[5]],
    [tag[1], tag[4], tag[5]],
    [tag[1], tag[4], tag[5]],
    [tag[1], tag[4], tag[5]],
    [tag[1], tag[5]],
    [tag[3], tag[4]],
    [tag[1], tag[5]],
    [tag[3]],
    [tag[3]],
    [tag[3]],
    [tag[1], tag[5]],
    [tag[3], tag[4]],
    [tag[3], tag[4]],
    [tag[1], tag[4], tag[5]],
    [tag[3]],
    [tag[1], tag[4], tag[6]],
    [tag[1], tag[4], tag[6]],
    [tag[1], tag[4], tag[6]],
    [tag[1], tag[4], tag[6]],
    [tag[1], tag[4], tag[6]],
    [tag[1], tag[4], tag[6]],
    [tag[1], tag[6]],
    [tag[1], tag[4], tag[6]],
    [tag[1], tag[4], tag[6]],
    [tag[1], tag[4], tag[6]],
    [tag[2]],
    [tag[1], tag[4], tag[6]],
    [tag[1], tag[4], tag[6]],
    [tag[1], tag[4], tag[5]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[2]],
    [tag[2]],
    [tag[2]],
    [tag[2]],
    [tag[2]],
    [tag[2]],
    [tag[2]],
    [tag[2]],
    [tag[2]],
    [tag[2]],
    [tag[2]],
    [tag[2]],
    [tag[0], tag[4]],
    [tag[0]],
    [tag[2]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0], tag[4]],
    [tag[0]],
    [tag[0]],
    [tag[2]],
]

### Tache 3 : Analyse de donnees

#### Objectif
Construire des profils de preferences utilisateur bases sur leurs selections d'images.

#### Ce que vous devez faire

1. **Simuler des utilisateurs** (creer au moins 5 utilisateurs) :
   - Chaque utilisateur met en "favori" 10 a 20 images
   - Les utilisateurs doivent avoir des preferences differentes (l'un aime la nature, un autre l'architecture, etc.)

2. **Construire les profils utilisateur** en analysant leurs images favorites :
   ```python
   user_profile = {
       "user_id": "user_001",
       "favorite_colors": ["bleu", "vert"],         # Couleurs les plus frequentes
       "favorite_orientation": "paysage",            # Orientation la plus frequente
       "favorite_size": "moyenne",                   # Taille la plus frequente
       "favorite_tags": ["nature", "eau"],            # Tags les plus frequents
       "favorite_images": ["img_01.jpg", ...]        # Liste des images favorites
   }
   ```

3. **Analyser les tendances** entre les utilisateurs :
   - Quelles couleurs sont les plus populaires globalement ?
   - Quels tags apparaissent le plus frequemment ?
   - Y a-t-il des groupes d'utilisateurs avec des preferences similaires ?

#### Resultat attendu
- `data/users.json` avec les profils utilisateur
- Resultats d'analyse montrant les tendances de preferences utilisateur

#### Conseils
- Utilisez pandas pour l'analyse de donnees (groupby, value_counts)
- Utilisez Counter du module collections pour trouver les elements les plus frequents
- Envisagez d'utiliser le regroupement pour grouper les utilisateurs similaires




In [ ]:
import json
import random
from collections import Counter
import pandas as pd
import numpy as np
from pathlib import Path
from PIL import Image
from sklearn.cluster import KMeans
import os

# ── Charger les métadonnées des images ──────────────────────────
data = json.load(open("data/image_metadata.json"))
dataframe = pd.json_normalize(data)

# ── Récupérer les tags définis dans la cellule précédente ───────
tag = ["album_photo", "extérieur", "illisible", "intérieur", "aJE", "jour", "nuit"]

photo_tags = [
    [tag[1], tag[4], tag[5]], [tag[1], tag[4], tag[5]], [tag[1], tag[4], tag[5]],
    [tag[1], tag[4], tag[5]], [tag[1], tag[4], tag[5]], [tag[1], tag[5]],
    [tag[3], tag[4]], [tag[3], tag[4]], [tag[3], tag[4]], [tag[3]], [tag[3]], [tag[3]],
    [tag[3], tag[4], tag[6]], [tag[3]], [tag[3]], [tag[3], tag[4]], [tag[3], tag[4]],
    [tag[3], tag[4]], [tag[3], tag[4], tag[6]], [tag[3], tag[4], tag[6]], [tag[1], tag[5]],
    [tag[3]], [tag[1], tag[5]], [tag[3]], [tag[3]], [tag[3]], [tag[3]], [tag[1], tag[5]],
    [tag[1], tag[4], tag[5]], [tag[1], tag[4], tag[5]], [tag[1], tag[4], tag[5]],
    [tag[1], tag[4], tag[5]], [tag[1], tag[5]], [tag[3], tag[4]], [tag[1], tag[5]],
    [tag[3]], [tag[3]], [tag[3]], [tag[1], tag[5]], [tag[3], tag[4]], [tag[3], tag[4]],
    [tag[1], tag[4], tag[5]], [tag[3]], [tag[1], tag[4], tag[6]], [tag[1], tag[4], tag[6]],
    [tag[1], tag[4], tag[6]], [tag[1], tag[4], tag[6]], [tag[1], tag[4], tag[6]],
    [tag[1], tag[4], tag[6]], [tag[1], tag[6]], [tag[1], tag[4], tag[6]],
    [tag[1], tag[4], tag[6]], [tag[1], tag[4], tag[6]], [tag[2]], [tag[1], tag[4], tag[6]],
    [tag[1], tag[4], tag[6]], [tag[1], tag[4], tag[5]], [tag[0]], [tag[0]], [tag[0]],
    [tag[0]], [tag[0]], [tag[0]], [tag[0]], [tag[0]], [tag[0]], [tag[0]], [tag[0]],
    [tag[0]], [tag[0]], [tag[0]], [tag[0]], [tag[0]], [tag[0]], [tag[0]], [tag[0]],
    [tag[0]], [tag[2]], [tag[2]], [tag[2]], [tag[2]], [tag[2]], [tag[2]], [tag[2]],
    [tag[2]], [tag[2]], [tag[2]], [tag[2]], [tag[2]], [tag[0], tag[4]], [tag[0]],
    [tag[2]], [tag[0]], [tag[0]], [tag[0]], [tag[0]], [tag[0], tag[4]], [tag[0]], [tag[0]], [tag[2]],
]

# Ajouter les tags au dataframe
dataframe["tags"] = photo_tags[:len(dataframe)]

# ── Fonction pour convertir RGB en nom de couleur (inspiré de TP2) ──
def rgb_to_color_name(r, g, b):
    """Convertit une couleur RGB en nom de couleur en français."""
    # Définir les couleurs de base avec leurs plages
    colors = {
        "rouge": (255, 0, 0),
        "vert": (0, 255, 0),
        "bleu": (0, 0, 255),
        "jaune": (255, 255, 0),
        "orange": (255, 165, 0),
        "violet": (128, 0, 128),
        "rose": (255, 192, 203),
        "marron": (139, 69, 19),
        "noir": (0, 0, 0),
        "blanc": (255, 255, 255),
        "gris": (128, 128, 128),
        "beige": (245, 245, 220),
        "cyan": (0, 255, 255),
    }
    
    min_distance = float('inf')
    closest_color = "gris"
    
    for name, (cr, cg, cb) in colors.items():
        distance = ((r - cr) ** 2 + (g - cg) ** 2 + (b - cb) ** 2) ** 0.5
        if distance < min_distance:
            min_distance = distance
            closest_color = name
    
    return closest_color

# ── Extraction des couleurs dominantes avec KMeans (TP2 Exercice 3) ──
def extract_dominant_colors(image_path, n_colors=3):
    """Extrait les couleurs dominantes d'une image avec KMeans."""
    try:
        img = Image.open(image_path)
        # Convertir en RGB si nécessaire
        if img.mode != 'RGB':
            img = img.convert('RGB')
        # Réduire la taille pour accélérer le traitement
        img.thumbnail((100, 100))
        pixels = np.array(img.getdata(), np.uint8)
        
        # Clustering KMeans
        kmeans = KMeans(n_clusters=n_colors, n_init=2, random_state=42)
        kmeans.fit(pixels)
        
        # Trier par fréquence
        labels, counts = np.unique(kmeans.labels_, return_counts=True)
        sorted_indices = np.argsort(counts)[::-1]
        
        # Convertir les centres en noms de couleurs
        color_names = []
        for idx in sorted_indices:
            center = kmeans.cluster_centers_[idx]
            color_name = rgb_to_color_name(int(center[0]), int(center[1]), int(center[2]))
            if color_name not in color_names:
                color_names.append(color_name)
        
        return color_names[:2]  # Top 2 couleurs uniques
    except Exception as e:
        return ["gris"]

# Extraire les couleurs pour toutes les images
print("Extraction des couleurs dominantes (KMeans)...")
dominant_colors = []
for idx, row in dataframe.iterrows():
    img_path = f"images/{row['id']}"
    if os.path.exists(img_path):
        colors = extract_dominant_colors(img_path)
    else:
        colors = ["gris"]
    dominant_colors.append(colors)
    if (idx + 1) % 20 == 0:
        print(f"  [{idx + 1}/{len(dataframe)}] images traitées...")

dataframe["dominant_colors"] = dominant_colors

# Charger orientation et size_category
try:
    enriched_data = json.load(open("data/metadata.json"))
    enriched_df = pd.json_normalize(enriched_data)
    dataframe["orientation"] = enriched_df["orientation"]
    dataframe["size_category"] = enriched_df["size_category"]
except FileNotFoundError:
    orientations = []
    sizes = []
    for _, row in dataframe.iterrows():
        img_path = f"images/{row['id']}"
        if os.path.exists(img_path):
            with Image.open(img_path) as img:
                w, h = img.size
                orientations.append("landscape" if w > h else ("portrait" if h > w else "square"))
                max_dim = max(w, h)
                sizes.append("thumbnail" if max_dim < 500 else ("medium" if max_dim <= 1500 else "large"))
        else:
            orientations.append("unknown")
            sizes.append("unknown")
    dataframe["orientation"] = orientations
    dataframe["size_category"] = sizes

print(f"\n✓ {len(dataframe)} images chargées avec tags, couleurs et métadonnées")
dataframe[["id", "tags", "dominant_colors", "orientation", "size_category"]].head(10)

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# 1. SIMULER DES UTILISATEURS avec préférences différentes
# ══════════════════════════════════════════════════════════════════════════════

random.seed(42)  # Pour reproductibilité

# Définir les profils de préférences pour 5 utilisateurs
user_preferences = {
    "user_001": {"preferred_tags": ["extérieur", "jour"], "preferred_orientation": "landscape"},
    "user_002": {"preferred_tags": ["intérieur", "aJE"], "preferred_orientation": "portrait"},
    "user_003": {"preferred_tags": ["album_photo"], "preferred_orientation": None},
    "user_004": {"preferred_tags": ["nuit", "extérieur"], "preferred_orientation": "landscape"},
    "user_005": {"preferred_tags": ["intérieur", "nuit"], "preferred_orientation": "portrait"},
}

def select_favorites_for_user(df, preferred_tags, preferred_orientation, num_favorites=15):
    """Sélectionne des images favorites basées sur les préférences utilisateur."""
    scores = []
    for idx, row in df.iterrows():
        score = 0
        image_tags = row.get("tags", [])
        # Score basé sur les tags préférés
        for tag in preferred_tags:
            if tag in image_tags:
                score += 2
        # Score basé sur l'orientation préférée
        if preferred_orientation and row.get("orientation") == preferred_orientation:
            score += 1
        scores.append((idx, score))
    
    # Trier par score décroissant et ajouter un peu d'aléatoire
    scores.sort(key=lambda x: (-x[1], random.random()))
    
    # Sélectionner entre 10 et 20 favorites
    num_to_select = random.randint(10, min(20, len(scores)))
    selected_indices = [idx for idx, _ in scores[:num_to_select]]
    
    return df.iloc[selected_indices]["id"].tolist()

# Créer les favoris pour chaque utilisateur
users = {}
for user_id, prefs in user_preferences.items():
    favorites = select_favorites_for_user(
        dataframe, 
        prefs["preferred_tags"], 
        prefs["preferred_orientation"]
    )
    users[user_id] = {"favorite_images": favorites}
    print(f"{user_id}: {len(favorites)} images favorites")

print(f"\n✓ {len(users)} utilisateurs simulés")

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# 2. CONSTRUIRE LES PROFILS UTILISATEUR
# ══════════════════════════════════════════════════════════════════════════════

def build_user_profile(user_id, favorite_images, df):
    """Construit un profil utilisateur à partir de ses images favorites."""
    # Filtrer le dataframe pour les images favorites
    favorites_df = df[df["id"].isin(favorite_images)]
    
    # Couleurs les plus fréquentes (extrait avec KMeans - TP2)
    all_colors = []
    for colors in favorites_df["dominant_colors"]:
        all_colors.extend(colors)
    color_counter = Counter(all_colors)
    favorite_colors = [color for color, _ in color_counter.most_common(2)]
    
    # Tags les plus fréquents
    all_tags = []
    for tags in favorites_df["tags"]:
        all_tags.extend(tags)
    tag_counter = Counter(all_tags)
    favorite_tags = [tag for tag, _ in tag_counter.most_common(3)]
    
    # Orientation la plus fréquente (value_counts pandas)
    orientation_counts = favorites_df["orientation"].value_counts()
    favorite_orientation = orientation_counts.index[0] if len(orientation_counts) > 0 else "unknown"
    
    # Taille la plus fréquente (value_counts pandas)
    size_counts = favorites_df["size_category"].value_counts()
    favorite_size = size_counts.index[0] if len(size_counts) > 0 else "unknown"
    
    return {
        "user_id": user_id,
        "favorite_colors": favorite_colors,
        "favorite_orientation": favorite_orientation,
        "favorite_size": favorite_size,
        "favorite_tags": favorite_tags,
        "favorite_images": favorite_images
    }

# Construire les profils pour tous les utilisateurs
user_profiles = []
for user_id, user_data in users.items():
    profile = build_user_profile(user_id, user_data["favorite_images"], dataframe)
    user_profiles.append(profile)
    print(f"\n{user_id}:")
    print(f"  Couleurs favorites: {profile['favorite_colors']}")
    print(f"  Orientation: {profile['favorite_orientation']}")
    print(f"  Taille: {profile['favorite_size']}")
    print(f"  Tags favoris: {profile['favorite_tags']}")
    print(f"  Nb images: {len(profile['favorite_images'])}")

# Sauvegarder les profils utilisateur
DATA_DIR = Path("data")
with open(DATA_DIR / "users.json", "w", encoding="utf-8") as f:
    json.dump(user_profiles, f, ensure_ascii=False, indent=2)

print(f"\n✓ Profils utilisateur sauvegardés dans data/users.json")

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# 3. ANALYSER LES TENDANCES ENTRE UTILISATEURS
# ══════════════════════════════════════════════════════════════════════════════

# ─── a) Couleurs les plus populaires globalement ───────────────────────────────
all_favorite_colors = []
for profile in user_profiles:
    all_favorite_colors.extend(profile["favorite_colors"])

global_color_counts = Counter(all_favorite_colors)
print("═" * 60)
print("COULEURS LES PLUS POPULAIRES (globalement)")
print("═" * 60)
for color, count in global_color_counts.most_common():
    print(f"  {color}: {count} utilisateurs")

# ─── b) Tags les plus populaires globalement ───────────────────────────────────
all_favorite_tags = []
for profile in user_profiles:
    all_favorite_tags.extend(profile["favorite_tags"])

global_tag_counts = Counter(all_favorite_tags)
print("\n" + "═" * 60)
print("TAGS LES PLUS POPULAIRES (globalement)")
print("═" * 60)
for tag, count in global_tag_counts.most_common():
    print(f"  {tag}: {count} utilisateurs")

# ─── c) Orientations préférées ─────────────────────────────────────────────────
orientation_counts = Counter([p["favorite_orientation"] for p in user_profiles])
print("\n" + "═" * 60)
print("ORIENTATIONS PRÉFÉRÉES")
print("═" * 60)
for orientation, count in orientation_counts.most_common():
    print(f"  {orientation}: {count} utilisateurs")

# ─── d) Tailles préférées ──────────────────────────────────────────────────────
size_counts = Counter([p["favorite_size"] for p in user_profiles])
print("\n" + "═" * 60)
print("TAILLES D'IMAGES PRÉFÉRÉES")
print("═" * 60)
for size, count in size_counts.most_common():
    print(f"  {size}: {count} utilisateurs")

In [ ]:
# ─── e) Groupes d'utilisateurs avec préférences similaires ────────────────────
print("\n" + "═" * 60)
print("GROUPES D'UTILISATEURS PAR PRÉFÉRENCES SIMILAIRES")
print("═" * 60)

def calculate_similarity(profile1, profile2):
    """Calcule le score de similarité entre deux profils."""
    score = 0
    # Similarité des couleurs
    common_colors = set(profile1["favorite_colors"]) & set(profile2["favorite_colors"])
    score += len(common_colors) * 2
    # Similarité des tags
    common_tags = set(profile1["favorite_tags"]) & set(profile2["favorite_tags"])
    score += len(common_tags) * 2
    # Similarité d'orientation
    if profile1["favorite_orientation"] == profile2["favorite_orientation"]:
        score += 1
    # Similarité de taille
    if profile1["favorite_size"] == profile2["favorite_size"]:
        score += 1
    return score

# Créer une matrice de similarité
print("\nMatrice de similarité entre utilisateurs:")
print("-" * 60)
header = "          " + "  ".join([p["user_id"][-3:] for p in user_profiles])
print(header)

for p1 in user_profiles:
    row = f"{p1['user_id'][-3:]}: "
    for p2 in user_profiles:
        sim = calculate_similarity(p1, p2)
        row += f"    {sim}" if p1 != p2 else "    -"
    print(row)

# Identifier les groupes
print("\nGroupes identifiés:")
print("-" * 60)

# Groupe extérieur/jour
outdoor_users = [p["user_id"] for p in user_profiles if "extérieur" in p["favorite_tags"]]
if outdoor_users:
    print(f"  📷 Amateurs d'extérieur: {', '.join(outdoor_users)}")

# Groupe intérieur
indoor_users = [p["user_id"] for p in user_profiles if "intérieur" in p["favorite_tags"]]
if indoor_users:
    print(f"  🏠 Amateurs d'intérieur: {', '.join(indoor_users)}")

# Groupe albums
album_users = [p["user_id"] for p in user_profiles if "album_photo" in p["favorite_tags"]]
if album_users:
    print(f"  📚 Amateurs d'albums: {', '.join(album_users)}")

# Groupe nuit
night_users = [p["user_id"] for p in user_profiles if "nuit" in p["favorite_tags"]]
if night_users:
    print(f"  🌙 Amateurs de photos de nuit: {', '.join(night_users)}")

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# RÉSUMÉ DES PROFILS UTILISATEUR
# ══════════════════════════════════════════════════════════════════════════════

# Créer un DataFrame récapitulatif
summary_data = []
for profile in user_profiles:
    summary_data.append({
        "Utilisateur": profile["user_id"],
        "Couleurs favorites": ", ".join(profile["favorite_colors"]),
        "Orientation": profile["favorite_orientation"],
        "Taille": profile["favorite_size"],
        "Tags favoris": ", ".join(profile["favorite_tags"]),
        "Nb favoris": len(profile["favorite_images"])
    })

summary_df = pd.DataFrame(summary_data)
print("═" * 80)
print("TABLEAU RÉCAPITULATIF DES PROFILS UTILISATEUR")
print("═" * 80)
summary_df